# Create Bottleneck features

---

Before supplying an image to a pre-trained network in Keras, there are some required preprocessing steps. When using TensorFlow as backend, Keras Convolutional Neural Networks (CNNs) require a 4D-array as input, with shape `(nb_images, rows, columns, channels)`, where `nb_images` corresponds to the total number of images, and rows, columns, and channels correspond to the number of `rows`, `columns`, and `channels` for each image, respectively.

The images from the training, validation and test datasets are first loaded.

In [2]:
from keras.applications.vgg16 import preprocess_input
import numpy as np
from preprocess import *

train_files, train_targets = load_dataset('data/preprocess/train.npz')
valid_files, valid_targets = load_dataset('data/preprocess/valid.npz')
test_files, test_targets = load_dataset('data/preprocess/test.npz')

print('There are %d images in the training dataset' % len(train_files))
print('There are %d images in the validation dataset' % len(valid_files))
print('There are %d images in the test dataset' % len(test_files))

There are 176131 images in the training dataset
There are 29355 images in the validation dataset
There are 29356 images in the test dataset


Each image is resized to a square image that is 224 x 224 pixels. Next, the image is converted to a 4D-array. Since, we are working with color images (`channels` = 3, RGB), the 4D-array has shape (1, 224, 224, 3).

The `paths_to_tensor` function used in the cell below takes a numpy array of string-valued image paths as input and returns a 4D-array with shape (`nb_images`, 224, 224, 3).

In [3]:
nb_train, nb_valid, nb_test = (30000,3000,3000)

train_tensors = preprocess_input(paths_to_tensor(train_files[:nb_train]))
valid_tensors = preprocess_input(paths_to_tensor(valid_files[:nb_valid]))
test_tensors = preprocess_input(paths_to_tensor(test_files[:nb_test]))

100%|██████████| 3000/3000 [00:09<00:00, 327.21it/s]


## 1. VGG16

### A. Import the VGG-16 Model, with the Final Fully-Connected Layers Removed

When performing transfer learning, we need to remove the final layers of the network, as they are too specific to the ImageNet database.  This is accomplished in the code cell below.

In [4]:
from keras.applications.vgg16 import VGG16

model = VGG16(include_top=False)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

### B. Extract Output of Final Max Pooling Layer

Now, the network stored in `model` is a truncated version of the VGG-16 network, where the final three fully-connected layers have been removed.  In this case, `model.predict` returns a 3D array (with dimensions 7 x 7 x 512) corresponding to the final max pooling layer of VGG-16.  The dimensionality of the obtained output from passing the 4D-array through the model is `(nb_images, 7, 7, 512)`.

In [5]:
train_vgg16 = model.predict(train_tensors)
valid_vgg16 = model.predict(valid_tensors)
test_vgg16  = model.predict(test_tensors)

In [6]:
np.savez('data/bottleneck_features/yelp_vgg16.npz', 
         train_features=train_vgg16, train_targets=train_targets[:nb_train],
         valid_features=valid_vgg16, valid_targets=valid_targets[:nb_valid],
         test_features=test_vgg16, test_targets=test_targets[:nb_test])

## 2. Xception
The same procedure is applied for the Xceotion deep learning model.

In [4]:
from keras.applications.xception import Xception

model = Xception(include_top=False)
model.summary()

83689472/83683744 [==============================] - 6s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
____________________________________________

In [5]:
train_xception = model.predict(train_tensors)
valid_xception = model.predict(valid_tensors)
test_xception  = model.predict(test_tensors)

In [ ]:
np.savez('data/bottleneck_features/yelp_xception.npz', 
         train_features=train_xception, train_targets=train_targets[:nb_train],
         valid_features=valid_xception, valid_targets=valid_targets[:nb_valid],
         test_features=test_xception, test_targets=test_targets[:nb_test])